In [4]:
import time
import pickle
import numpy as np
import os
from generate_landmark import get_landmark

In [12]:
def convert_to_frames(video_path, frames_path):
    command = "/home/luqman/ffmpeg/ffmpeg-git-20220622-amd64-static/ffmpeg -loglevel warning -t 15 -i " + video_path + " -r:v 30 -frames:v 450 " + os.path.join(frames_path, "%d.png") 
    os.system(command)
    
def delete_dir(path):
    command = "rm -r " + path
    os.system(command)
    
def make_prediction(frames_path, out_path):
    frames_count = len([x for x in os.listdir(frames_path)])
    landmarks = []
    
    for i in range(1, frames_count+1):
        filename = str(i) + '.png'
        img_path = os.path.join(frames_path, filename)
        landmark_list = get_landmark(img_path)
        if (landmark_list is not None):
            landmarks.append(landmark_list[0])
    
    landmarks = np.array(landmarks)
    with open(out_path, 'wb') as f:
        np.save(f, landmarks)

In [11]:
start_time = time.time()
video_dir = '/nas/database/Big5/train'
frames_dir = "/nas/project_data/B1_Behavior/hakim/FI/frames/train"
landmark_path = '/nas/project_data/B1_Behavior/hakim/FI/3DDFA_feature/train'
annotation_path = '/nas/database/Big5/gt/annotation_training.pkl'

with open(annotation_path, 'rb') as f:
    annotation = pickle.load(f, encoding='latin1')
videos = list(annotation['extraversion'].keys())

for i, video in enumerate(videos):
    video_path = os.path.join(video_dir, video)
    video_name = video.rsplit(".")[0] + "." + video.rsplit(".")[1]
    output_name = video_name + '.npy'
    out_path = os.path.join(landmark_path, output_name)
    frames_path = os.path.join(frames_dir, video_name)
    
    if (i < 1000 or os.path.exists(out_path)):
        continue
    elif (i == 1000):
        break
    
    print(f'Processing {i} {out_path}')
    os.makedirs(frames_path, exist_ok=True)
    convert_to_frames(video_path, frames_path)
    make_prediction(frames_path, out_path)
    delete_dir(frames_path)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.012633085250854492 seconds ---


In [ ]:
# annotation_path = '/nas/database/Big5/gt/annotation_validation.pkl'

In [64]:
PATH = '/nas/project_data/B1_Behavior/hakim/FI/3DDFA_feature/val/modNfUPt3F4.002.npy'
with open(PATH, 'rb') as f:
    o = np.load(f, allow_pickle=True)

In [65]:
o.shape

(450, 3, 68)